# Spark DataFrames Project Exercise 

#### Import needed libraries

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, min, max, corr, year, month

#### Start a simple Spark Session

In [0]:
spark = SparkSession.builder.appName("WalmartStock").getOrCreate()

#### Load the data, have Spark infer the data types.

In [0]:
df = spark.read.csv("data/walmart_stock.csv", inferSchema=True, header=True)

#### What are the column names?

In [0]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [0]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Show first 5 columns.

In [0]:
df.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows


#### Use describe() to learn about the DataFrame.

In [0]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev| 6.768090244708252|6.768186808159206|6.744075756255481|6.756859163733001|4519780.843155604|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

#### Create a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [0]:
df.withColumn("HV Ratio", df["High"]/df["Volume"]).select("HV Ratio").show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows


#### What day had the Peak High in Price?

In [0]:
df.orderBy("High").sort(df["High"].desc()).show(1)

+----------+---------+---------+-----+---------+-------+---------+
|      Date|     Open|     High|  Low|    Close| Volume|Adj Close|
+----------+---------+---------+-----+---------+-------+---------+
|2015-01-13|90.800003|90.970001|88.93|89.309998|8215400|83.825448|
+----------+---------+---------+-----+---------+-------+---------+
only showing top 1 row


#### What is the mean of the Close column?

In [0]:
df.select(mean("Close")).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [0]:
df.select(max("Volume"), min("Volume")).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [0]:
df.filter(df["Close"] < 60).count()

81

#### What percentage of the time was the High greater than 80 dollars ?

In [0]:
df.filter("High > 80").count() / df.count() * 100

9.141494435612083

#### What is the Pearson correlation between High and Volume?

In [0]:
df.select(corr("High","Volume")).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### What is the max High per year?

In [0]:
df.groupBy(year("Date")).max("High").orderBy(year("Date")).show()

+----------+---------+
|year(Date)|max(High)|
+----------+---------+
|      2012|77.599998|
|      2013|81.370003|
|      2014|88.089996|
|      2015|90.970001|
|      2016|75.190002|
+----------+---------+



#### What is the average Close for each Calendar Month?

In [0]:
df.groupBy(month("Date")).avg("Close").orderBy(month("Date")).show()

+-----------+-----------------+
|month(Date)|       avg(Close)|
+-----------+-----------------+
|          1|71.44801958415842|
|          2|  71.306804443299|
|          3|71.77794377570092|
|          4|72.97361900952382|
|          5|72.30971688679247|
|          6| 72.4953774245283|
|          7|74.43971943925233|
|          8|73.02981855454546|
|          9|72.18411785294116|
|         10|71.57854545454543|
|         11| 72.1110893069307|
|         12|72.84792478301885|
+-----------+-----------------+

